In [18]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.features import shapes
import os
import requests
import zipfile

In [19]:
file_path = './heat_risk_analysis_Day+1_20240805.geoparquet'
gdf = gpd.read_parquet(file_path)

In [20]:
gdf.head(3)

geometry  raster_value  \
0  POLYGON ((-95.13265 49.35965, -95.13265 49.181...           0.0   
1  POLYGON ((-122.05381 49.0167, -122.05381 48.98...           2.0   
2  POLYGON ((-121.59752 49.0167, -121.59752 48.98...           0.0   

   weighted_ALAND20  weighted_AWATER20  weighted_STATEFP10  weighted_GEOID10  \
0       323659886.0       1.219293e+09                27.0         2756711.0   
1       130928393.0       9.274130e+05                53.0         5398266.0   
2       860753189.0       1.665025e+07                53.0         5398844.0   

   weighted_MULTI_STATE  weighted_POP  weighted_PR_HRI  weighted_F_HRI  ...  \
0                   0.0          96.0           -999.0          -999.0  ...   
1                   0.0        3642.0             20.0             0.5  ...   
2                   0.0        4740.0             20.0             0.5  ...   

   weighted_OVERALL_RANK  mode_ZCTA5CE20  mode_AFFGEOID20  mode_GEOID20  \
0                 0.8957           56711   860Z200US56711         56711   
1                 0.8204           98266   860Z200US98266         98266   
2                 0.9722           98844   860Z200US98844         98844   

   mode_NAME20  mode_LSAD20  mode_STATE  mode_STATE_ABV  mode_ZCTA  highlight  
0        56711           Z5          MN       Minnesota      56711      False  
1        98266           Z5          WA      Washington      98266       True  
2        98844           Z5          WA      Washington      98844      False  

[3 rows x 85 columns]

In [23]:
highlighted_states_counts = gdf[gdf['highlight']]['mode_STATE_ABV'].value_counts()
highlighted_states_df = highlighted_states_counts.reset_index()
highlighted_states_df.columns = ["State", "Cells"]
highlighted_states_df = highlighted_states_df.sort_values("Cells", ascending=False)
highlighted_states_df

State  Cells
0                 Oregon    292
1                 Nevada    269
2             California    242
3                  Idaho    208
4             Washington    176
5               Colorado    108
6               Michigan    105
7               New York     93
8                   Utah     89
9                   Ohio     76
10               Indiana     64
11          Pennsylvania     54
12               Montana     50
13              Missouri     50
15              Kentucky     47
14         West Virginia     47
16              Illinois     42
17              Nebraska     41
18                Kansas     28
19              Virginia     26
20               Wyoming     22
21             Minnesota     22
22              Maryland     20
23                  Iowa     16
24          North Dakota     14
25          South Dakota     13
26  District of Columbia      2
27              Delaware      1

In [16]:
gdf.columns

Index(['geometry', 'raster_value', 'weighted_ALAND20', 'weighted_AWATER20',
       'weighted_STATEFP10', 'weighted_GEOID10', 'weighted_MULTI_STATE',
       'weighted_POP', 'weighted_PR_HRI', 'weighted_F_HRI', 'weighted_LOW_EMS',
       'weighted_P_NEHD', 'weighted_PR_NEHD', 'weighted_HHB_SCORE',
       'weighted_HHB_RANK', 'weighted_P_CHD', 'weighted_PR_CHD',
       'weighted_F_CHD', 'weighted_P_OBS', 'weighted_PR_OBS', 'weighted_F_OBS',
       'weighted_P_DIABETES', 'weighted_PR_DIABETES', 'weighted_F_DIABETES',
       'weighted_P_COPD', 'weighted_PR_COPD', 'weighted_F_COPD',
       'weighted_P_ASTHMA', 'weighted_PR_ASTHMA', 'weighted_F_ASTHMA',
       'weighted_P_MNTHL', 'weighted_PR_MNTHL', 'weighted_F_MNTHL',
       'weighted_F_SEN_COUNT', 'weighted_SEN_SCORE', 'weighted_SEN_RANK',
       'weighted_P_UNINSUR', 'weighted_PR_UNINSUR', 'weighted_P_POV',
       'weighted_PR_POV', 'weighted_P_UNEMP', 'weighted_PR_UNEMP',
       'weighted_P_NOHSDP', 'weighted_PR_NOHSDP', 'weighted_P_ISO'

In [9]:
import pandas as pd

# Assuming your DataFrame is named df and it has a column 'weighted_GEOID10'
df = gdf.copy()
# Calculate the length of each cell
df['weighted_GEOID10'] = df['weighted_GEOID10'].astype(str)
df['length'] = df['weighted_GEOID10'].apply(len)

# Count the occurrences of each length
length_counts = df['length'].value_counts().sort_index()



In [12]:
df[df['length']==8]

geometry  raster_value  \
11068  POLYGON ((-110.7378 39.17141, -110.7378 39.136...           2.0   
11092  POLYGON ((-123.05765 39.38334, -123.05765 39.3...           2.0   
11101  POLYGON ((-117.97001 39.13603, -117.97001 39.1...           1.0   
11121  POLYGON ((-117.90157 39.15372, -117.90157 39.1...           1.0   
11122  POLYGON ((-118.19816 39.13603, -118.19816 39.1...           2.0   
...                                                  ...           ...   
23106  POLYGON ((-80.18913 25.69087, -80.18913 25.670...           2.0   
23137  POLYGON ((-81.01045 24.74142, -81.01045 24.720...           3.0   
23138  POLYGON ((-80.98764 24.74142, -80.98764 24.720...           2.0   
23148  POLYGON ((-81.69489 24.59629, -81.69489 24.575...           4.0   
23149  POLYGON ((-81.80896 24.61704, -81.80896 24.575...           3.0   

       weighted_ALAND20  weighted_AWATER20  weighted_STATEFP10  \
11068       365819539.0           651144.0                 6.0   
11092       571384627.0          5278962.0                 6.0   
11101       135061971.0           153823.0                 6.0   
11121       161881042.0          1447386.0                 6.0   
11122       669933796.0          2062823.0                 6.0   
...                 ...                ...                 ...   
23106       375846947.0           254961.0                 8.0   
23137       394963210.0           143387.0                 6.0   
23138       154608815.0            93504.0                 6.0   
23148       231661592.0          1880021.0                 6.0   
23149       142887682.0            12723.0                 6.0   

      weighted_GEOID10  weighted_MULTI_STATE  weighted_POP  weighted_PR_HRI  \
11068         695543.0                   0.0         765.0             20.0   
11092         696044.0                   0.0        1022.0             97.0   
11101         697635.0                   1.0         289.0             20.0   
11121         695567.0                   0.0        1930.0             20.0   
11122         696050.0                   0.0         414.0             20.0   
...                ...                   ...           ...              ...   
23106         881128.0                   0.0         115.0             20.0   
23137         693643.0                   0.0        3171.0             72.0   
23138         693645.0                   0.0         318.0             20.0   
23148         695033.0                   0.0        9372.0             47.0   
23149         695006.0                   0.0        8979.0             42.0   

       weighted_F_HRI  ...  mode_ZCTA5CE20  mode_AFFGEOID20  mode_GEOID20  \
11068            0.50  ...           95543   860Z200US95543         95543   
11092            1.00  ...           96044   860Z200US96044         96044   
11101            0.50  ...           97635   860Z200US97635         97635   
11121            0.50  ...           95567   860Z200US95567         95567   
11122            0.50  ...           96050   860Z200US96050         96050   
...               ...  ...             ...              ...           ...   
23106            0.50  ...           81128   860Z200US81128         81128   
23137            0.75  ...           93643   860Z200US93643         93643   
23138            0.50  ...           93645   860Z200US93645         93645   
23148            0.50  ...           95033   860Z200US95033         95033   
23149            0.50  ...           95006   860Z200US95006         95006   

       mode_NAME20  mode_LSAD20  mode_STATE  mode_STATE_ABV  mode_ZCTA  \
11068        95543           Z5          CA      California      95543   
11092        96044           Z5          CA      California      96044   
11101        97635           Z5          CA      California      97635   
11121        95567           Z5          CA      California      95567   
11122        96050           Z5          CA      California      96050   
...            ...          ...         ...     

In [7]:
gdf.to_csv('temp.csv')

In [4]:
gdf.columns

Index(['geometry', 'raster_value', 'weighted_ALAND20', 'weighted_AWATER20',
       'weighted_STATEFP10', 'weighted_GEOID10', 'weighted_MULTI_STATE',
       'weighted_POP', 'weighted_PR_HRI', 'weighted_F_HRI', 'weighted_LOW_EMS',
       'weighted_P_NEHD', 'weighted_PR_NEHD', 'weighted_HHB_SCORE',
       'weighted_HHB_RANK', 'weighted_P_CHD', 'weighted_PR_CHD',
       'weighted_F_CHD', 'weighted_P_OBS', 'weighted_PR_OBS', 'weighted_F_OBS',
       'weighted_P_DIABETES', 'weighted_PR_DIABETES', 'weighted_F_DIABETES',
       'weighted_P_COPD', 'weighted_PR_COPD', 'weighted_F_COPD',
       'weighted_P_ASTHMA', 'weighted_PR_ASTHMA', 'weighted_F_ASTHMA',
       'weighted_P_MNTHL', 'weighted_PR_MNTHL', 'weighted_F_MNTHL',
       'weighted_F_SEN_COUNT', 'weighted_SEN_SCORE', 'weighted_SEN_RANK',
       'weighted_P_UNINSUR', 'weighted_PR_UNINSUR', 'weighted_P_POV',
       'weighted_PR_POV', 'weighted_P_UNEMP', 'weighted_PR_UNEMP',
       'weighted_P_NOHSDP', 'weighted_PR_NOHSDP', 'weighted_P_ISO'

In [13]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 23150 entries, 0 to 23149
Data columns (total 85 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   geometry                 23150 non-null  geometry
 1   raster_value             23150 non-null  float64 
 2   weighted_ALAND20         23150 non-null  float64 
 3   weighted_AWATER20        23150 non-null  float64 
 4   weighted_STATEFP10       23150 non-null  float64 
 5   weighted_GEOID10         23150 non-null  float64 
 6   weighted_MULTI_STATE     23150 non-null  float64 
 7   weighted_POP             23150 non-null  float64 
 8   weighted_PR_HRI          23150 non-null  float64 
 9   weighted_F_HRI           23150 non-null  float64 
 10  weighted_LOW_EMS         23150 non-null  float64 
 11  weighted_P_NEHD          23150 non-null  float64 
 12  weighted_PR_NEHD         23150 non-null  float64 
 13  weighted_HHB_SCORE       23150 non-null  float64 
 14

In [20]:
file_1 = './data/heat_risk_analysis_Day 1_20240731_with_county_state.parquet'
gdf_1 = gpd.read_parquet(file_1)

In [22]:
gdf_1.head()

geometry  raster_value  \
0  POLYGON ((-95.10984 49.35965, -95.10984 49.344...           4.0   
1  POLYGON ((-95.01858 49.35965, -95.01858 49.344...           4.0   
2  POLYGON ((-94.92732 49.34479, -94.92732 49.329...           2.0   
3  POLYGON ((-94.92732 49.32993, -94.92732 49.300...           3.0   
4  POLYGON ((-95.13265 49.35965, -95.13265 49.240...           3.0   

   weighted_ALAND20  weighted_AWATER20  weighted_STATEFP10  weighted_GEOID10  \
0       323659886.0       1.219293e+09                27.0         2756711.0   
1       323659886.0       1.219293e+09                27.0         2756711.0   
2       323659886.0       1.219293e+09                27.0         2756711.0   
3       323659886.0       1.219293e+09                27.0         2756711.0   
4       323659886.0       1.219293e+09                27.0         2756711.0   

   weighted_MULTI_STATE  weighted_POP  weighted_PR_HRI  weighted_F_HRI  ...  \
0                   0.0          96.0           -999.0          -999.0  ...   
1                   0.0          96.0           -999.0          -999.0  ...   
2                   0.0          96.0           -999.0          -999.0  ...   
3                   0.0          96.0           -999.0          -999.0  ...   
4                   0.0          96.0           -999.0          -999.0  ...   

   mode_STATE_NAME  mode_LSAD_1  mode_STATEFP_2  mode_STATENS  \
0        Minnesota           06              27      00662849   
1        Minnesota           06              27      00662849   
2        Minnesota           06              27      00662849   
3        Minnesota           06              27      00662849   
4        Minnesota           06              27      00662849   

   mode_AFFGEOID_2  mode_GEOID_2  mode_STUSPS_2  mode_NAME_2  mode_LSAD_2  \
0      0400000US27            27             MN    Minnesota           00   
1      0400000US27            27             MN    Minnesota           00   
2      0400000US27            27             MN    Minnesota           00   
3      0400000US27            27             MN    Minnesota           00   
4      0400000US27            27             MN    Minnesota           00   

   highlight  
0       True  
1       True  
2       True  
3       True  
4       True  

[5 rows x 106 columns]

In [25]:
gdf_1.columns[:50]

Index(['geometry', 'raster_value', 'weighted_ALAND20', 'weighted_AWATER20',
       'weighted_STATEFP10', 'weighted_GEOID10', 'weighted_MULTI_STATE',
       'weighted_POP', 'weighted_PR_HRI', 'weighted_F_HRI', 'weighted_LOW_EMS',
       'weighted_P_NEHD', 'weighted_PR_NEHD', 'weighted_HHB_SCORE',
       'weighted_HHB_RANK', 'weighted_P_CHD', 'weighted_PR_CHD',
       'weighted_F_CHD', 'weighted_P_OBS', 'weighted_PR_OBS', 'weighted_F_OBS',
       'weighted_P_DIABETES', 'weighted_PR_DIABETES', 'weighted_F_DIABETES',
       'weighted_P_COPD', 'weighted_PR_COPD', 'weighted_F_COPD',
       'weighted_P_ASTHMA', 'weighted_PR_ASTHMA', 'weighted_F_ASTHMA',
       'weighted_P_MNTHL', 'weighted_PR_MNTHL', 'weighted_F_MNTHL',
       'weighted_F_SEN_COUNT', 'weighted_SEN_SCORE', 'weighted_SEN_RANK',
       'weighted_P_UNINSUR', 'weighted_PR_UNINSUR', 'weighted_P_POV',
       'weighted_PR_POV', 'weighted_P_UNEMP', 'weighted_PR_UNEMP',
       'weighted_P_NOHSDP', 'weighted_PR_NOHSDP', 'weighted_P_ISO'